In [1]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plta# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add project root to path
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add project root to path
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))


In [2]:
# Import our analysis modules
from scripts.data_fetcher import DarkPoolDataFetcher


In [3]:
# Initialize data fetchers
dark_pool_fetcher = DarkPoolDataFetcher()
options_fetcher = OptionsDataFetcher()

NameError: name 'OptionsDataFetcher' is not defined

In [ ]:
# Fetch recent dark pool trades
dark_pool_trades = dark_pool_fetcher.fetch_recent_trades(date_str="2024-04-09")
print(f"Fetched {len(dark_pool_trades)} dark pool trades")

In [ ]:
# Third cell - Analysis
# Display dark pool trade summary
print("\nDark Pool Trade Summary:")
print(f"Total Trades: {len(dark_pool_trades)}")

if not dark_pool_trades.empty:
    # Basic statistics
    print("\nBasic Statistics:")
    print(f"Unique Symbols: {len(dark_pool_trades['ticker'].unique())}")
    print(f"Total Volume: {dark_pool_trades['size'].sum():,.0f}")
    print(f"Total Premium: ${dark_pool_trades['premium'].sum():,.2f}")
    
    # Block trades analysis
    print("\nSignificant Trades:")
    print(f"Block Trades: {dark_pool_trades['is_block_trade'].sum()}")
    print(f"High Premium Trades: {dark_pool_trades['is_high_premium'].sum()}")
    print(f"Price Impact Trades: {dark_pool_trades['is_price_impact'].sum()}")
    
    # Per symbol breakdown
    print("\nPer Symbol Breakdown:")
    for symbol in dark_pool_trades['ticker'].unique():
        symbol_trades = dark_pool_trades[dark_pool_trades['ticker'] == symbol]
        print(f"\n{symbol}:")
        print(f"  Number of Trades: {len(symbol_trades)}")
        print(f"  Total Volume: {symbol_trades['size'].sum():,.0f}")
        print(f"  Average Trade Size: {symbol_trades['size'].mean():,.0f}")
        print(f"  Total Premium: ${symbol_trades['premium'].sum():,.2f}")
        print(f"  Average Price Impact: {symbol_trades['price_impact'].mean():.4%}")

In [ ]:
# Visualization cell
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style and color palette
sns.set_theme(style="whitegrid")
sns.set_palette("husl")

# Create subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

# 1. Volume by Symbol
sns.barplot(data=dark_pool_trades.groupby('ticker')['size'].sum().reset_index(),
           x='size', y='ticker', ax=ax1)
ax1.set_title('Total Volume by Symbol', fontsize=12)
ax1.set_xlabel('Volume', fontsize=10)
ax1.set_ylabel('Symbol', fontsize=10)

# 2. Trade Size Distribution
sns.histplot(data=dark_pool_trades, x='size', bins=30, ax=ax2)
ax2.set_title('Distribution of Trade Sizes', fontsize=12)
ax2.set_xlabel('Trade Size', fontsize=10)
ax2.set_ylabel('Count', fontsize=10)

# 3. Price Impact by Symbol
sns.barplot(data=dark_pool_trades.groupby('ticker')['price_impact'].mean().reset_index(),
           x='price_impact', y='ticker', ax=ax3)
ax3.set_title('Average Price Impact by Symbol', fontsize=12)
ax3.set_xlabel('Price Impact', fontsize=10)
ax3.set_ylabel('Symbol', fontsize=10)

# 4. Time Series of Trades
dark_pool_trades['timestamp'] = pd.to_datetime(dark_pool_trades['timestamp'])
for symbol in dark_pool_trades['ticker'].unique():
    symbol_data = dark_pool_trades[dark_pool_trades['ticker'] == symbol]
    sns.lineplot(data=symbol_data, 
                x='timestamp', 
                y=symbol_data['size'].cumsum(),
                label=symbol,
                ax=ax4)
ax4.set_title('Cumulative Volume Over Time by Symbol', fontsize=12)
ax4.set_xlabel('Time', fontsize=10)
ax4.set_ylabel('Cumulative Volume', fontsize=10)
ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout and display
plt.tight_layout()
plt.show()

# Additional detailed plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Box plot of trade sizes by symbol
sns.boxplot(data=dark_pool_trades, x='ticker', y='size', ax=ax1)
ax1.set_title('Trade Size Distribution by Symbol', fontsize=12)
ax1.set_xlabel('Symbol', fontsize=10)
ax1.set_ylabel('Trade Size', fontsize=10)
ax1.tick_params(rotation=45)

# Violin plot of price impact by symbol
sns.violinplot(data=dark_pool_trades, x='ticker', y='price_impact', ax=ax2)
ax2.set_title('Price Impact Distribution by Symbol', fontsize=12)
ax2.set_xlabel('Symbol', fontsize=10)
ax2.set_ylabel('Price Impact', fontsize=10)
ax2.tick_params(rotation=45)

plt.tight_layout()
plt.show()

# Heatmap of correlations
plt.figure(figsize=(12, 8))
numeric_cols = ['size', 'price', 'premium', 'price_impact', 'nbbo_mid']
correlation_matrix = dark_pool_trades[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap of Trade Metrics', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Fifth cell - Interactive table display
# Display the trades in a sortable table format
from IPython.display import display
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Create a summary table
summary_df = dark_pool_trades.groupby('ticker').agg({
    'size': ['count', 'sum', 'mean'],
    'premium': ['sum', 'mean'],
    'price_impact': 'mean',
    'is_block_trade': 'sum'
}).round(2)

# Rename columns for clarity
summary_df.columns = ['Trade Count', 'Total Volume', 'Avg Size', 
                     'Total Premium', 'Avg Premium', 'Avg Price Impact',
                     'Block Trade Count']

display(summary_df)

In [ ]:
# 1. Detailed SPY/QQQ Comparison Analysis
def analyze_symbol_comparison(trades_df):
    comparison = {}
    
    for symbol in ['SPY', 'QQQ']:
        symbol_trades = trades_df[trades_df['ticker'] == symbol]
        
        # Calculate key metrics
        comparison[symbol] = {
            'trade_count': len(symbol_trades),
            'total_volume': symbol_trades['size'].sum(),
            'avg_trade_size': symbol_trades['size'].mean(),
            'median_trade_size': symbol_trades['size'].median(),
            'total_premium': symbol_trades['premium'].sum(),
            'avg_premium': symbol_trades['premium'].mean(),
            'avg_price_impact': symbol_trades['price_impact'].mean(),
            'block_trades': symbol_trades['is_block_trade'].sum(),
            'block_trade_ratio': symbol_trades['is_block_trade'].mean(),
            'high_premium_ratio': symbol_trades['is_high_premium'].mean(),
            'price_impact_ratio': symbol_trades['is_price_impact'].mean()
        }
    
    # Convert to DataFrame for easy comparison
    comparison_df = pd.DataFrame(comparison).round(4)
    
    print("SPY vs QQQ Comparison:\n")
    print(comparison_df)
    
    # Calculate relative metrics
    print("\nRelative Metrics (SPY/QQQ):")
    relative_metrics = comparison_df['SPY'] / comparison_df['QQQ']
    print(relative_metrics.round(2))

analyze_symbol_comparison(dark_pool_trades)

In [ ]:
# 2. SPY/QQQ Focused Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# Set style
sns.set_theme(style="whitegrid")

# Create figure with subplots
fig = plt.figure(figsize=(20, 15))

# 1. Trade Size Distribution Comparison
plt.subplot(2, 2, 1)
sns.boxplot(data=dark_pool_trades, x='ticker', y='size', showfliers=False)
plt.title('Trade Size Distribution')
plt.yscale('log')

# 2. Price Impact Comparison
plt.subplot(2, 2, 2)
sns.violinplot(data=dark_pool_trades, x='ticker', y='price_impact')
plt.title('Price Impact Distribution')

# 3. Cumulative Volume Over Time
plt.subplot(2, 2, 3)
for symbol in ['SPY', 'QQQ']:
    symbol_data = dark_pool_trades[dark_pool_trades['ticker'] == symbol]
    plt.plot(symbol_data['timestamp'], 
             symbol_data['size'].cumsum(), 
             label=symbol)
plt.title('Cumulative Volume')
plt.legend()
plt.xticks(rotation=45)

# 4. Block Trade Proportion
plt.subplot(2, 2, 4)
block_trade_props = dark_pool_trades.groupby('ticker')['is_block_trade'].value_counts(normalize=True).unstack()
block_trade_props.plot(kind='bar', stacked=True)
plt.title('Block Trade Proportion')
plt.legend(['Regular Trades', 'Block Trades'])

plt.tight_layout()
plt.show()

In [ ]:
# 3. Time-Based Analysis
# Convert timestamp to datetime if not already
dark_pool_trades['timestamp'] = pd.to_datetime(dark_pool_trades['timestamp'])

# Add time-based features
dark_pool_trades['hour'] = dark_pool_trades['timestamp'].dt.hour
dark_pool_trades['minute'] = dark_pool_trades['timestamp'].dt.minute
dark_pool_trades['day_of_week'] = dark_pool_trades['timestamp'].dt.day_name()

# Create time-based visualizations
fig = plt.figure(figsize=(20, 15))

# 1. Hourly Volume Distribution
plt.subplot(2, 2, 1)
hourly_volume = dark_pool_trades.groupby(['ticker', 'hour'])['size'].sum().unstack(0)
sns.heatmap(hourly_volume.T, cmap='YlOrRd', annot=True, fmt='.0f')
plt.title('Hourly Trading Volume')
plt.xlabel('Hour of Day')

# 2. Average Trade Size by Hour
plt.subplot(2, 2, 2)
for symbol in ['SPY', 'QQQ']:
    symbol_data = dark_pool_trades[dark_pool_trades['ticker'] == symbol]
    hourly_avg = symbol_data.groupby('hour')['size'].mean()
    plt.plot(hourly_avg.index, hourly_avg.values, label=symbol, marker='o')
plt.title('Average Trade Size by Hour')
plt.legend()
plt.grid(True)

# 3. Price Impact by Hour
plt.subplot(2, 2, 3)
hourly_impact = dark_pool_trades.groupby(['ticker', 'hour'])['price_impact'].mean().unstack()
sns.heatmap(hourly_impact.T, cmap='coolwarm', annot=True, fmt='.4f')
plt.title('Average Price Impact by Hour')
plt.xlabel('Hour of Day')

# 4. Block Trade Frequency by Hour
plt.subplot(2, 2, 4)
block_trades = dark_pool_trades[dark_pool_trades['is_block_trade']]
hourly_blocks = block_trades.groupby(['ticker', 'hour']).size().unstack(0).fillna(0)
sns.heatmap(hourly_blocks.T, cmap='YlOrRd', annot=True, fmt='.0f')
plt.title('Block Trade Frequency by Hour')
plt.xlabel('Hour of Day')

plt.tight_layout()
plt.show()

# Print time-based statistics
print("\nTime-Based Statistics:")
for symbol in ['SPY', 'QQQ']:
    symbol_data = dark_pool_trades[dark_pool_trades['ticker'] == symbol]
    print(f"\n{symbol} Trading Patterns:")
    
    # Busiest hour
    busiest_hour = symbol_data.groupby('hour')['size'].sum().idxmax()
    print(f"Busiest Hour: {busiest_hour}:00")
    
    # Hour with largest average trades
    largest_trades_hour = symbol_data.groupby('hour')['size'].mean().idxmax()
    print(f"Hour with Largest Avg Trades: {largest_trades_hour}:00")
    
    # Hour with highest price impact
    highest_impact_hour = symbol_data.groupby('hour')['price_impact'].mean().idxmax()
    print(f"Hour with Highest Price Impact: {highest_impact_hour}:00")